# Simple generation of signals

In [1]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt


import gc
import sys

import os
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)

#from teproteus import TEProteusAdmin, TEProteusInst
#from tevisainst import TEVisaInst
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst

import mylib as ml

####
import time
import math
import ipywidgets as widgets
# from scipy import signal

# Parameters of the signal generator (DAC)

In [2]:
SCLK = 2500e6  ### sampling clock
DUC_INTERP = 1  ### the DUC mode

# Parameters of the digitizer (ADC)

In [10]:
DIG_SCLK = 2700e6
DUC_NCO = 1040e6 #MHz
DDC_NCO = 240e6
DUC_INTERP = 8

In [3]:
# # Connect to instrument via PXI
# admin = TepAdmin()   

# # Get list of available PXI slots
# slot_ids = admin.get_slot_ids()

# # Assume that at least one slot was found
# sid = slot_ids[0]  # this is the system ID, the port trough which we access the Proteus

# # Open a single-slot instrument:
# inst = admin.open_instrument(slot_id=sid)

In [4]:
# def connection_func ():
    
#     print("\n===CONNECTING===")

#         # Connect to instrument via PXI
#     admin = TepAdmin()   

#     # Get list of available PXI slots
#     slot_ids = admin.get_slot_ids()

#     # Assume that at least one slot was found
#     sid = slot_ids[0]  # this is the system ID, the port trough which we access the Proteus

#     # Open a single-slot instrument:
#     inst = admin.open_instrument(slot_id=sid)

#     # Get the instrument's *IDN
#     resp = inst.send_scpi_query('*IDN?')
#     print('Connected to: ' + resp)

#     # Get the model name
#     model = inst.send_scpi_query(":SYST:iNF:MODel?")
#     print("Model: " + model)

#     # Infer the natural DAC waveform format
#     if 'P9082' in model:
#         dac_mode = 8
#     else:
#         dac_mode = 16
#     print("DAC waveform format: {0} bits-per-point".format(dac_mode))

#     if dac_mode == 16:
#         max_dac = 65535  # = (2**16 - 1)  ,because the momory slots are 16bit ones (have 16bit vertical resolution) 
#         data_type = np.uint16 
#     else:
#         max_dac = 255
#         data_type = np.uint8 

#     half_dac = max_dac / 2.0
    
#     print("===CONNECTION ESTABLISHED===")
    
#     return dac_mode, max_dac, sid, data_type

# Connect to the instrument via PXI connection

In [5]:
dac_mode, max_dac, sid, data_type = ml.connection_func() # get the DAC mode, range, Serial port # and the data type of the system (8 or 16bit)
# ml.system_info_func(max_dac, sid)  # prints out some parameters of the system that might be of interest
# Make sure the Model name is P9484M, that the name of our module


=========CONNECTING=========
process id 18864
Connected to: Tabor Electronics,P9484M,000000220642,1.118.0
Model: P9484M
===CONNECTION ESTABLISHED===

===SYSTEM'S INFORMATION===
DAC waveform format: 16 bits-per-point
Max DAC wave-point level: 65535
Half DAC wave-point level: 32767.0
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 8,581,545,984 wave-bytes
==========READY============


# Initialize channels

In [6]:
admin = TepAdmin()
inst = admin.open_instrument(slot_id=sid)

In [8]:
type(inst)

teproteus.TEProteusInst

In [9]:
channel_numb = 1
ml.initialization_func(channel_numb, sid, SCLK)

channel_numb = 3 ### there are only channels 1 and 3 accessable (easily), actually there are 4, of which two are masked and two are acecessable
ml.initialization_func(channel_numb, sid, SCLK)


===INITIALIZATION OF CHANNEL 1===  -->  ===DONE===
ERRORS: 0, no error

===INITIALIZATION OF CHANNEL 3===  -->  ===DONE===
ERRORS: 0, no error


# Define a Sinusoid and a Blank (0V) Pulse

In [10]:
sin_f, sin_f_seglen = ml.sin_func (1000, 0, SCLK, show_plot = False)  ### creating a sinus shape
blank, blank_seglen = ml.blank_func (SCLK)

sin_f = ml.digital_conv_func(sin_f, max_dac, data_type) ### converting the signal from float format array to a binary format array
blank = ml.digital_conv_func(blank, max_dac, data_type)

original seglen: 1000000.0
formated seglen: 1000000


# Define a gaussian Pulse

In [11]:
# def prepare_gaussian_pulse (sigma, width_over_sigma, frequency, SCLK, DUC_INTERP, max_dac, show_plot = True):
    
#     half_dac = max_dac / 2
    
# #     frequency *= 2.5

#     SCLK = SCLK/2.5

#     period = 1/frequency
#     delta_t = 1/SCLK ### how much time between each two sequential signal points
#     sigma_numerical = sigma/delta_t  ### how many datapoints between each two sequential signal points

#     print ("period = {0}[ns]".format(period*1e9))
#     print ("sigma = {0}[ns]".format(sigma*1e9))
#     print ("sigma_num = {0} datapoints ".format(sigma_numerical))

#     width = width_over_sigma*sigma_numerical # time width of the gaussian pulse, the width of the pulse terminates at 5*sigma, this is made by convention, it can be change to be (arbitrary number*sigma)
#     print (width_over_sigma, sigma_numerical, width)
#     seglen_gauss = int(width)
#     print("Gaussian Segement length = {0} datapoints\n".format(seglen_gauss))
#     print("Gaussian Segement length (in time) = {0} [ns]]".format(width_over_sigma*sigma*1e9))

# ################# WARNINGS AND CHECKS #######################

#     if 5*period > sigma:
#         print ("\n!WARNING! Sigma is comparable to the period of the oscillation\n")
    
#     seglen_gauss, normalization_factor = ml.formatter(seglen_gauss)

# #     seglen_gauss_old = seglen_gauss
# #     n = -32.0 + seglen_gauss/64.
    
# #     if seglen_gauss < 2048:
        
# #         print ("\n!WARNING! The segment of the defined signal ({0} points) is less then the minium necessary length for a Proteus signal (2048 points)".format(seglen_gauss))
# #         print ("-----------Will edit the signal automatically to reach 2048 points (~82[ns])-----------\n")
        
# #         seglen_gauss *= int(2048 / seglen_gauss)
# #         seglen_gauss += int(2048 % seglen_gauss)
# #         normalization_factor = seglen_gauss/seglen_gauss_old #to make the gausian from [0 to x] to [0 to 1] 
        
# #         print("\nGaussian Segement length (after correction) = {0} datapoints".format(seglen_gauss))
        
# #     if n.is_integer() == False:
        
# #         print ("\n!WARNING! The segment of the defined signal ({0} points) does not comply with the format of the possible data chunk for a signal, which is datalength = ( 64 * (32 + n)), where n is an integer ".format(seglen_gauss))
# #         print ("-----------Will edit the signal automatically to suit the # of points-----------\n")
        
# #         n = int(n)
# #         seglen_gauss = 64 * (32 + n)
# #         normalization_factor = seglen_gauss/seglen_gauss_old #to make the gausian from [0 to x] to [0 to 1] 
# #         print("\nGaussian Segement length (after correction) = {0} datapoints".format(seglen_gauss))
        
# #     else:
# #         normalization_factor = 1


        
# ############################################################
    
# ###########################################################

#     t = np.linspace(-1, 1, seglen_gauss, endpoint=False)
    
#     ss = sigma_numerical / seglen_gauss

#     GAUS_FC = frequency * seglen_gauss * DUC_INTERP /2 / SCLK
    
#     print('Gaussian frequency = {0}[Mhz]'.format(SCLK * 2 * GAUS_FC / seglen_gauss / 1e6/ DUC_INTERP))  # the actual frequency of the sin wave in the gaussian

#     ####################
#     sin = np.sin(2*np.pi*t*GAUS_FC)
#     cos = np.cos(2*np.pi*t*GAUS_FC)
#     gaussian = (1/ss/np.sqrt(2*np.pi)/2) * np.exp(-(t**2)/2/(ss**2)) / normalization_factor
#     (i) = sin*gaussian
#     (q) = cos*gaussian
#     ####################
    
#     if show_plot == True:
#         plt.plot(t, (i), '-',t, (q), '-')
#         plt.legend(['I','Q'])



    
#     return (i), (q), seglen_gauss


# COS DOO----------------------------------


# Do two plots one for the chip one for the detector
# - datapoints should be bigger
# -remoce the title from the plot
# - on Fig.6 make the flux arrows to look like cos law

In [12]:
frequency = 10e6 ### frequency of the oscillation
sigma = 346e-08  ### make it bigger then the period of the oscillation 
width_over_sigma = 5    # time width of the gaussian pulse, a.k.a. how many sigmas wide is the pulse, for example - width = [5*sigma], by convention

In [13]:
gauss_signal_i, gauss_signal_q, seglen_gauss = ml.prepare_gaussian_pulse (sigma, width_over_sigma, frequency, SCLK, DUC_INTERP, max_dac, show_plot = False)

gauss_signal_i = ml.digital_conv_func(gauss_signal_i, max_dac, data_type)
gauss_signal_q = ml.digital_conv_func(gauss_signal_q, max_dac, data_type)

period = 100.0[ns]
sigma = 3460.0[ns]
sigma_num = 3459.9999999999995 datapoints 
5 3459.9999999999995 17299.999999999996
Gaussian Segement length = 17299 datapoints

Gaussian Segement length (in time) = 17300.0 [ns]]

!WARNING! The segment of the defined signal (17299 points) does not comply with the format of the possible data chunk for a signal, which is ||| datalength = ( 64 * (32 + n)) |||, where         n is an integer.
-----------Will edit the signal automatically to suit the # of points-----------

Gaussian frequency = 10.0[Mhz]


In [17]:
channb = 1

segnum = 1
ml.download_signal(segnum, sin_f_seglen, channb, sin_f, sid) ### download signal to an approprate 'channel' and 'segment of the channel'

segnum = 2
ml.download_signal(segnum, sin_f_seglen, channb, sin_f, sid) #ml.download_signal(segnum,blank_seglen,channb,blank,sid) 

segnum = 3
ml.download_signal(segnum,seglen_gauss,channb,gauss_signal_i,sid)

0, no error 1 downloaded
0, no error 2 downloaded
0, no error 3 downloaded


In [ ]:
#############

In [ ]:
##########

# Load task table

In [ ]:
channb = 1

tasklen = 3
loop = 3

In [ ]:
################################### 

# Tasks definitions

In [20]:
tasklen = 1

#Select channel
cmd = ':INST:CHAN 1'
inst.send_scpi_cmd(cmd )

cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
inst.send_scpi_cmd(cmd )



for i in range(tasklen):
    curr_task = i+1
    cmd = ':TASK:COMP:SEL {0}'.format(curr_task)
    inst.send_scpi_cmd(cmd)
    cmd = ':TASK:COMP:DTRigger ON'
    inst.send_scpi_cmd(cmd)
 
    if curr_task == 1:  
        cmd = ':TASK:COMP:ENAB CPU'
        
    if curr_task==tasklen:
        cmd = ':TASK:COMP:NEXT1 {0}'.format(1)
        inst.send_scpi_cmd(cmd)
    else:
        cmd = ':TASK:COMP:NEXT1 {0}'.format(curr_task+1)
        inst.send_scpi_cmd(cmd )    

    cmd = ':TASK:COMP:TYPE SING'
    inst.send_scpi_cmd(cmd )

    cmd = ':TASK:COMP:LOOP {0}'.format(1)
    inst.send_scpi_cmd(cmd )

    cmd = ':TASK:COMP:SEGM {0}'.format(1)
    inst.send_scpi_cmd(cmd )

    
cmd = ':TASK:COMP:WRIT'
inst.send_scpi_cmd(cmd)
print('Downloading Task table to channel {0}'.format(1))

cmd = ':INST:CHAN {0}'.format(1)
inst.send_scpi_cmd(cmd)
    
cmd = ':OUTP ON'
inst.send_scpi_cmd(cmd)

cmd = 'FUNC:MODE TASK'
inst.send_scpi_cmd(cmd)


resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)


In [ ]:
# tasklen = 1

# admin = TepAdmin()  ### decalring a shortcut to send SCPY commands to the machine
# inst = admin.open_instrument(slot_id=sid)  ### decalring a shortcut to send SCPY commands to the machine
# num_channels = 1

# for ii in range(num_channels):
#     channb=ii+1

#     #Select channel
#     cmd = ':INST:CHAN {0}'.format(channb)
#     inst.send_scpi_cmd(cmd )

#     #define the length of the task table
#     cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
#     inst.send_scpi_cmd(cmd )

#     for jj in range(tasklen):
#         curr_task = jj+1
#         loop = jj+1
#         segnb = jj+1
#         # let us select the first task on the task table
#         cmd = ':TASK:COMP:SELect {0}'.format(curr_task)
#         inst.send_scpi_cmd(cmd)
#         print(curr_task,'=====',cmd)

#         # it will be a single type of task (it would do but 1 thing, not a sequence of things)
#         cmd = ':TASK:COMP:TYPE SING'
#         inst.send_scpi_cmd(cmd)

#         # lets choose how mamy times this task will repeat itself before terminating (how many times it would loop)
#         cmd = ':TASK:COMP:LOOP {0}'.format(loop)
#         inst.send_scpi_cmd(cmd)

#         # Choose now the segment from the memory to assign for this task
#         cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
#         inst.send_scpi_cmd(cmd)

#         # Here we loop the task sequance again and again until forever
#         if curr_task==tasklen:
#             cmd = ':TASK:COMP:NEXT1 {0}'.format(1)
#             inst.send_scpi_cmd(cmd)
#             print(curr_task,'=====',cmd,"final")
#         else:
#             cmd = ':TASK:COMP:NEXT1 {0}'.format(curr_task+1)
#             inst.send_scpi_cmd(cmd)
#             print(curr_task,'=====',cmd)


#     cmd = ':TASK:COMP:WRIT'
#     inst.send_scpi_cmd(cmd)
#     print('Downloading Task table of channel {0}'.format(channb))

#     resp = inst.send_scpi_query(':SYST:ERR?')
#     print(resp)


1 ===== :TASK:COMP:SELect 1
1 ===== :TASK:COMP:NEXT1 1 final
0, no error
1 ===== :TASK:COMP:SELect 1
1 ===== :TASK:COMP:NEXT1 1 final
0, no error
1 ===== :TASK:COMP:SELect 1
1 ===== :TASK:COMP:NEXT1 1 final
0, no error
1 ===== :TASK:COMP:SELect 1
1 ===== :TASK:COMP:NEXT1 1 final
0, no error


# Switch to task mode

In [ ]:

# for ii in range(num_channels):

#     channb=ii+1


#     cmd = ':INST:CHAN {0}'.format(channb)
#     inst.send_scpi_cmd(cmd)

#     cmd = ':OUTP ON'
#     inst.send_scpi_cmd(cmd)

#     cmd = 'FUNC:MODE TASK'
#     inst.send_scpi_cmd(cmd)

#     resp = inst.send_scpi_query(':SYST:ERR?')
#     print(resp)


# if dac_mode==16:

#     for jj in range(2):

#         mrk = jj+1

#         cmd = ':MARK:SEL {0}'.format(mrk)  # why do we need marks?
#         inst.send_scpi_cmd(cmd)

#         cmd = ':MARK ON'
#         inst.send_scpi_cmd(cmd)

# elif dac_mode==8:

#     for jj in range(4):

#         mrk = jj+1

#         cmd = ':MARK:SEL {0}'.format(mrk)
#         inst.send_scpi_cmd(cmd)

#         cmd = ':MARK ON'
#         inst.send_scpi_cmd(cmd)   
            
        

0, no error
0, no error
0, no error
0, no error


In [ ]:
###################################

# FIX THE READOUT

In [ ]:
# admin = TepAdmin()   
# inst = admin.open_instrument(slot_id=sid)

# #Select channel
# cmd = ':INST:CHAN {0}'.format(channb)
# inst.send_scpi_cmd(cmd )

# #define the length of the task table
# cmd = ':TASK:COMP:LENG {0}'.format(tasklen)
# inst.send_scpi_cmd(cmd )
    
# ######Set the delay in the channel in [seconds]
# delay_time=4e-9

# #__________________________________The Black box computation
# delay_samples=SCLK*delay_time
# custom_loops= delay_samples/10 # we will loop a segment of 100 samples a "custom_loops" number of times
# #delay_samples_integer=delay_samples//65536
# #delay_samples_remnant=delay_samples%65536
# print(custom_loops)
# #__________________________________
# ######
    
# for jj in range(tasklen):
#     curr_task = jj+1
#     #loop = jj+1
#     segnb = jj+1
#     # let us select the first task on the task table
#     cmd = ':TASK:COMP:SEL {0}'.format(curr_task)
#     inst.send_scpi_cmd(cmd)

#     # it will be a single type of task (it would do ust 1 thing not a sequence of things)
#     cmd = ':TASK:COMP:TYPE SING'
#     inst.send_scpi_cmd(cmd)

#     # lets choose how mamy times this task will repeat itself before terminating (how many times it would loop)
#     cmd = ':TASK:COMP:LOOP {0}'.format(loop)
#     inst.send_scpi_cmd(cmd)

#     # Choose now the segment from the memory to assign for this task
#     cmd = ':TASK:COMP:SEGM {0}'.format(segnb)
#     inst.send_scpi_cmd(cmd)

#     # Here we loop the task sequance again and again until forever
#     if curr_task==2:
#  #       cmd = ":TASK:COMP:DEL 50000"
#         cmd = ":TASK:COMP:LOOP {0}".format(custom_loops) 
#         inst.send_scpi_cmd(cmd)
#  #       inst.send_scpi_cmd(cmd)
#     if curr_task==tasklen:
#         cmd = ':TASK:COMP:NEXT1 {0}'.format(1)
#         inst.send_scpi_cmd(cmd)
#     else:
#         cmd = ':TASK:COMP:NEXT1 {0}'.format(curr_task+1)
#         inst.send_scpi_cmd(cmd)


# cmd = ':TASK:COMP:WRIT'
# inst.send_scpi_cmd(cmd )
# print('Downloading Task table of channel {0}'.format(channb))

# resp = inst.send_scpi_query(':SYST:ERR?')
# print(resp)

1.0
1 ===== :TASK:COMP:SELect 1
1 ===== :TASK:COMP:NEXT1 1 final
0, no error


# Shut down

In [21]:
# Disconnect
inst.close_instrument()
admin.close_inst_admin()

OSError: exception: access violation writing 0xFFFFFFFFFFFFFFFF